<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/multi_ticker_lightgbn_live_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Remove conflicting CUDA first
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y && apt-get clean

#Install compatible RAPIDS + LightGBM
!pip install lightgbm==4.1.0
!pip install --extra-index-url=https://pypi.nvidia.com \
    cupy-cuda12x cudf-cu12==24.4.0 cuml-cu12==24.4.0 dask-cudf-cu12==24.4.0


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [1]:
!pip install pandas numpy yfinance matplotlib scikit-learn lightgbm==4.1.0 joblib

In [3]:
!pip install tensorflow==2.18.0

In [2]:
!pip install yfinance pandas numpy matplotlib scikit-learn lightgbm xgboost joblib

In [4]:
!pip install stable-baselines3 gymnasium gym-anytrading

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [21]:
!rm -rf /content/drive


In [22]:
# === Imports ===
import os
import gc
import time
import joblib
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation
from google.colab import drive

# === MOUNT GOOGLE DRIVE ===
drive.mount('/content/drive')

# === Save Paths ===
SAVE_DIR = "/content/drive/MyDrive/Results_May_2025/results_lightgbn_walkforward/lightgbn_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/results_lightgbn_walkforward"
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/plots", exist_ok=True)

# === Configuration ===
TEST_MODE = False
TICKERS = ['AAPL'] if TEST_MODE else [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL']

required_cols = ['Close', 'SMA_50', 'EMA_20', 'RSI', 'MACD', 'Signal_Line', 'ATR', 'OBV', 'CCI']

# === Feature Engineering ===
def compute_technical_indicators(df):
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
    df['ATR'] = df['High'].rolling(window=14).max() - df['Low'].rolling(window=14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    df.dropna(inplace=True)
    return df

# === Label Generation ===
def generate_labels(df):
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    return df.dropna()

# === Walkforward Training + Evaluation ===
def run_lgbm_walkforward(ticker, window_size=3000, step_size=500, initial_cash=100000):
    print(f"\nRunning walkforward LGBM for {ticker}")

    test_end_dt = datetime.today()
    train_start_dt = test_end_dt - timedelta(days=729)
    train_end_dt = train_start_dt + timedelta(days=365)

    train_start = train_start_dt.strftime("%Y-%m-%d")
    test_end = test_end_dt.strftime("%Y-%m-%d")

    try:
        df = yf.download(ticker, start=train_start, end=test_end, interval="1h", progress=False)
    except Exception as e:
        print(f"Download failed for {ticker}: {e}")
        return

    if df.empty:
        print("Empty dataframe. Skipping.")
        return

    df = compute_technical_indicators(df)
    if df.empty or not all(col in df.columns for col in required_cols):
        print("Missing required columns after feature engineering.")
        return
    df = generate_labels(df)

    scaler = MinMaxScaler()
    features = required_cols

    portfolio_lgbm = []
    portfolio_hold = []
    close_tracker = []
    capital = initial_cash
    shares = initial_cash / df['Close'].iloc[0]
    y_pred_all = []

    for start in range(0, len(df) - window_size, step_size):
        end = start + window_size
        segment = df.iloc[start:end].copy()
        if segment.shape[0] < window_size:
            break

        X = segment[features]
        y = segment['Target']
        X_scaled = scaler.fit_transform(X)
        X_train, X_test = X_scaled[:-step_size], X_scaled[-step_size:]
        y_train, y_test = y[:-step_size], y[-step_size:]

        model = LGBMClassifier(n_estimators=500, learning_rate=0.05, random_state=42)
        model.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  callbacks=[early_stopping(20), log_evaluation(0)])

        preds = (model.predict(X_test) > 0.5).astype(int)
        y_pred_all.extend(preds)

        price_segment = segment['Close'].values[-step_size:]
        close_tracker.extend(price_segment)

        for i, pred in enumerate(preds):
            price = price_segment[i]
            if pred:
                capital = shares * price
            else:
                shares = capital / price
            print(f"Segment {start}-{end} | preds: {len(preds)} | cap: {float(capital):.2f} | shares: {float(shares):.2f}")
            portfolio_lgbm.append(float(capital))
            portfolio_hold.append(float(shares * price))

    if len(portfolio_lgbm) < 2 or len(y_pred_all) == 0 or len(close_tracker) < 2:
        print(f"\u26a0\ufe0f Skipping {ticker} — not enough data for evaluation.")
        return

    close_tracker = np.array(close_tracker).flatten()
    y_pred_all = np.array(y_pred_all).flatten()
    true_labels = (np.diff(close_tracker) > 0).astype(int)

    min_len = min(len(true_labels), len(y_pred_all))
    if min_len == 0:
        print(f"\u26a0\ufe0f Skipping accuracy for {ticker} — mismatched prediction lengths.")
        return

    accuracy = np.mean(y_pred_all[:min_len] == true_labels[:min_len])
    precision = precision_score(true_labels[:min_len], y_pred_all[:min_len])
    recall = recall_score(true_labels[:min_len], y_pred_all[:min_len])
    f1 = f1_score(true_labels[:min_len], y_pred_all[:min_len])
    returns = np.diff(portfolio_lgbm) / (np.array(portfolio_lgbm[:-1]) + 1e-6)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-6) * np.sqrt(252)
    drawdown = np.max(np.maximum.accumulate(portfolio_lgbm) - portfolio_lgbm)

    final_portfolio = portfolio_lgbm[-1]
    final_hold = portfolio_hold[-1]

    print(f"\n🔍 {ticker} Debug Info:")
    print(f"Final Portfolio: {final_portfolio:.2f} | Buy&Hold: {final_hold:.2f}")
    print(f"Sharpe: {sharpe:.4f} | Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f} | Drawdown: {drawdown:.2f}")

    model.booster_.save_model(f"{SAVE_DIR}/lgbm_model_{ticker}.txt")
    joblib.dump(scaler, f"{SAVE_DIR}/scaler_{ticker}.pkl")
    with open(f"{SAVE_DIR}/features_{ticker}.txt", "w") as f:
        f.write("\n".join(features))

    metrics_df = pd.DataFrame([{
        "Ticker": ticker,
        "Final_Portfolio": round(final_portfolio, 2),
        "Final_Hold": round(final_hold, 2),
        "Return_%": round((final_portfolio - initial_cash) / initial_cash * 100, 2),
        "Return_Hold_%": round((final_hold - initial_cash) / initial_cash * 100, 2),
        "Sharpe": round(sharpe, 4),
        "Accuracy": round(accuracy, 4),
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1_Score": round(f1, 4),
        "Drawdown": round(drawdown, 2)
    }])
    metrics_df.to_csv(f"{RESULTS_DIR}/metrics_{ticker}.csv", index=False)

    summary_path = os.path.join(RESULTS_DIR, "lightgbm_walkforward_summary.csv")
    if os.path.exists(summary_path):
        all_metrics = pd.read_csv(summary_path)
        all_metrics = all_metrics[all_metrics["Ticker"] != ticker]
        all_metrics = pd.concat([all_metrics, metrics_df], ignore_index=True)
    else:
        all_metrics = metrics_df
    all_metrics.to_csv(summary_path, index=False)

    plt.figure(figsize=(12, 6))
    plt.plot(portfolio_lgbm, label="LGBM Strategy")
    plt.plot(portfolio_hold, label="Buy & Hold")
    plt.title(f"{ticker} Portfolio Value")
    plt.xlabel("Step")
    plt.ylabel("Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{ticker}_portfolio_plot.png")
    plt.close()

    print(f"✅ Saved model, scaler, metrics, and plot for {ticker}")

# === Run ===
for ticker in TICKERS:
    run_lgbm_walkforward(ticker)
    gc.collect()
    time.sleep(1)


Mounted at /content/drive

Running walkforward LGBM for AAPL


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
<ipython-input-22-c472dda606b0>:127: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Segment {start}-{end} | preds: {len(preds)} | cap: {float(capital):.2f} | shares: {float(shares):.2f}")
<ipython-input-22-c472dda606b0>:128: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  portfolio_lgbm.append(float(capital))
<ipython-input-22-c472dda606b0>:129: FutureWarning: Calling float on a single element Series is 

[LightGBM] [Info] Number of positive: 1295, number of negative: 1205
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000333 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.518000 -> initscore=0.072031
[LightGBM] [Info] Start training from score 0.072031
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[4]	valid_0's binary_logloss: 0.690709
Segment 0-3000 | preds: 500 | cap: 123553.92 | shares: 541.10
Segment 0-3000 | preds: 500 | cap: 122566.42 | shares: 541.10
Segment 0-3000 | preds: 500 | cap: 122385.15 | shares: 541.10
Segment 0-3000 | preds: 500 | cap: 122747.74 | shares: 541.10
Segment 0-3000 | preds: 500 | cap: 122727.83 | shares: 541.10
Segment 0-3000 | preds: 500 | cap: 122726.04 | shares: 541.10
Segme

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
<ipython-input-22-c472dda606b0>:127: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Segment {start}-{end} | preds: {len(preds)} | cap: {float(capital):.2f} | shares: {float(shares):.2f}")
<ipython-input-22-c472dda606b0>:128: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  portfolio_lgbm.append(float(capital))
<ipython-input-22-c472dda606b0>:129: FutureWarning: Calling float on a single element Series is 

[LightGBM] [Info] Number of positive: 1269, number of negative: 1231
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.507600 -> initscore=0.030402
[LightGBM] [Info] Start training from score 0.030402
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.692608
Segment 0-3000 | preds: 500 | cap: 131839.66 | shares: 381.04
Segment 0-3000 | preds: 500 | cap: 131839.66 | shares: 391.04
Segment 0-3000 | preds: 500 | cap: 131839.66 | shares: 393.35
Segment 0-3000 | preds: 500 | cap: 131839.66 | shares: 389.99
Segment 0-3000 | preds: 500 | cap: 132982.32 | shares: 389.99
Segment 0-3000 | preds: 500 | cap: 132001.51 | shares: 389.99
Segme

...

[LightGBM] [Info] Number of positive: 1304, number of negative: 1196
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000097 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.521600 -> initscore=0.086454
[LightGBM] [Info] Start training from score 0.086454
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[3]	valid_0's binary_logloss: 0.690555
Segment 0-3000 | preds: 500 | cap: 214739.53 | shares: 1209.46
Segment 0-3000 | preds: 500 | cap: 214412.97 | shares: 1209.46
Segment 0-3000 | preds: 500 | cap: 213009.99 | shares: 1209.46
Segment 0-3000 | preds: 500 | cap: 213613.15 | shares: 1209.46
Segment 0-3000 | preds: 500 | cap: 214086.41 | shares: 1209.4

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
<ipython-input-22-c472dda606b0>:127: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Segment {start}-{end} | preds: {len(preds)} | cap: {float(capital):.2f} | shares: {float(shares):.2f}")
<ipython-input-22-c472dda606b0>:129: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Depr

[LightGBM] [Info] Number of positive: 1208, number of negative: 1292
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000324 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2294
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.483200 -> initscore=-0.067225
[LightGBM] [Info] Start training from score -0.067225
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.69611
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 1016.67
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 1027.38
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 1025.59
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 1024.38
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 1024.09
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 1025.1

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
<ipython-input-22-c472dda606b0>:127: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Segment {start}-{end} | preds: {len(preds)} | cap: {float(capital):.2f} | shares: {float(shares):.2f}")
<ipython-input-22-c472dda606b0>:129: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Depr

[LightGBM] [Info] Number of positive: 1292, number of negative: 1208
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000345 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2294
[LightGBM] [Info] Number of data points in the train set: 2500, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.516800 -> initscore=0.067225
[LightGBM] [Info] Start training from score 0.067225
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[34]	valid_0's binary_logloss: 0.692327
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 529.35
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 529.77
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 531.43
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 529.21
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 530.15
Segment 0-3000 | preds: 500 | cap: 100000.00 | shares: 529.24
Segm